In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter   # count the number of instances of each class
import time      # time library to calculate the time taken

In [2]:

# Load the data
train = pd.read_csv('train.csv')
validation = pd.read_csv('validation.csv')
test = pd.read_csv('test.csv')


In [3]:
#####Preprocessing (Train dataset)#####
# check for missing values and drop if have
if train.isnull().sum().sum() > 0:
    train = train.dropna()
    print(train.isnull().sum())
else: 
    print("No missing values in the dataset")


#check for duplicate values and drop if have
if train.duplicated().sum() > 0:
    train = train.drop_duplicates()
    print(train.duplicated().sum())
else:
    print("No duplicate values in the dataset")


#drop the loanID column as it is not required
if 'LoanID' in train.columns:
    train = train.drop(['LoanID'], axis=1)
    print("LoanID column has been removed")
else:
    print('LoanID column is not present or has removed, check the columns')


No missing values in the dataset
No duplicate values in the dataset
LoanID column has been removed


In [4]:
#converting CATEGORICAL variable to numerical/binary FOR TRAIN DATA
train['Education'] = train['Education'].map({'High School':1,'Bachelor\'s':2, 'Master\'s':3, 'PhD':4}).astype(int)
train['EmploymentType'] = train['EmploymentType'].map({'Unemployed':1,'Part-time':2, 'Full-time':3, 'Self-employed':4}).astype(int)
train['MaritalStatus']= train['MaritalStatus'].map({'Single':1,'Married':2, 'Divorced':3}).astype(int)
train['HasMortgage']= train['HasMortgage'].map({'No':0,'Yes':1}).astype(int)
train['HasDependents']= train['HasDependents'].map({'No':0,'Yes':1}).astype(int)
train['LoanPurpose']= train['LoanPurpose'].map({'Education':1,'Home':2, 'Business':3, 'Auto':4, 'Other':5}).astype(int)
train['HasCoSigner']= train['HasCoSigner'].map({'No':0,'Yes':1}).astype(int)

#convert the loan terms from months to years, reducing the scale
train['LoanTerm']= (train['LoanTerm']/12).astype(int)

print("Done converting categorical variables to numerical for train data")


#normalizing the data using Z-Score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_independent_variables = ['Age','Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', \
                                'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']

for column in train_independent_variables:
    train[column] = scaler.fit_transform(train[[column]])
    train[column] = train[column].apply(lambda x: round(x, 4))

print("Done Z-Score the data for train data")

#one hot encoding for categorical variables
columns_encode = ['Education', 'EmploymentType', 'MaritalStatus', 'LoanPurpose']
train = pd.get_dummies(train, columns=columns_encode)

#check and convert features boolean type to 1 and 0
train_columns =train.columns
for column in train_columns:
    if train[column].dtype == bool:
        train[column] = train[column].astype(int)


Done converting categorical variables to numerical for train data
Done Z-Score the data for train data


In [5]:

# library for oversampling, undersampling and combination of both, pick one of them
# Documentation: https://imbalanced-learn.org/dev/references/over_sampling.html

# Note: balancing the data for the minority class "Default=1"
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler, SMOTENC
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTEENN       #SMOTEENN is a combination of SMOTE and Edited Nearest Neighbours(ENN).


sm = SMOTE(sampling_strategy='minority',random_state=42)

#separate the data into Xs and Y
X_train = train.drop(['Default'], axis=1)
Y_train = train['Default']


print(Counter(Y_train))

X_train_resampled, Y_train_resampled = sm.fit_resample(X_train, Y_train)
train_resampled = pd.concat([X_train_resampled, Y_train_resampled], axis=1)

print(Counter(Y_train_resampled))
print("Done resampling for train data")

#separate the data into Xs and Y from resampled data
X_train = X_train_resampled
Y_train = Y_train_resampled 


#overwriting the train data with resampled data
train = train_resampled



#remove duplicate rows
if train.duplicated().sum() > 0:
    train = train.drop_duplicates()
    print(train.duplicated().sum())
else:
    print("No duplicate values in the dataset")



Counter({0: 135408, 1: 17788})
Counter({0: 135408, 1: 135408})
Done resampling for train data
No duplicate values in the dataset


In [6]:

# feature reduction, why we need it? - to reduce the features/dimension of the model
# VIF for each features in the train dataset
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(len(X_train.columns))]
print(vif_data)

#drop the columns if VIF > *your optimum value*
VIF_VALUE = 3
if vif_data['VIF'].max() > VIF_VALUE:
    vif_data = vif_data[vif_data['VIF'] > VIF_VALUE]
    X_train = X_train.drop(vif_data['feature'], axis=1)
    print(X_train.columns)
    print(vif_data['feature'] + " has been removed from the dataset") 
else: 
    print(X_train.columns)


#combine the Xs and Ys back
train = pd.concat([X_train, Y_train], axis=1)


#export (train) to csv
train.to_csv('train_preprocessed.csv', index=False)


             feature       VIF
0                Age  1.042137
1             Income  1.018082
2         LoanAmount  1.016647
3        CreditScore  1.002426
4     MonthsEmployed  1.014512
5     NumCreditLines  1.001596
6       InterestRate  1.026063
7           LoanTerm  1.000445
8           DTIRatio  1.000930
9        HasMortgage  1.761855
10     HasDependents  1.746044
11       HasCoSigner  1.738945
12       Education_1  2.715058
13       Education_2  2.697545
14       Education_3  2.622482
15       Education_4  2.611281
16  EmploymentType_1  2.794758
17  EmploymentType_2  2.706543
18  EmploymentType_3  2.575802
19  EmploymentType_4  2.671660
20   MaritalStatus_1  3.743785
21   MaritalStatus_2  3.666009
22   MaritalStatus_3  3.823498
23     LoanPurpose_1  2.128395
24     LoanPurpose_2  2.089041
25     LoanPurpose_3  2.154885
26     LoanPurpose_4  2.129116
27     LoanPurpose_5  2.126086
Index(['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
       'NumCreditLines', 'Inte

In [7]:
# validation data
if 'LoanID' in validation.columns:
    validation = validation.drop(['LoanID'], axis=1)
else:
    print('LoanID column is not present or has removed, check the columns')


#converting CATEGORICAL variable to numerical/binary FOR VALIDATION DATA
validation['Education'] = validation['Education'].map({'High School':1,'Bachelor\'s':2, 'Master\'s':3, 'PhD':4}).astype(int)
validation['EmploymentType'] = validation['EmploymentType'].map({'Unemployed':1,'Part-time':2, 'Full-time':3, 'Self-employed':4}).astype(int)
validation['MaritalStatus']= validation['MaritalStatus'].map({'Single':1,'Married':2, 'Divorced':3}).astype(int)
validation['HasMortgage']= validation['HasMortgage'].map({'No':0,'Yes':1}).astype(int)
validation['HasDependents']= validation['HasDependents'].map({'No':0,'Yes':1}).astype(int)
validation['LoanPurpose']= validation['LoanPurpose'].map({'Education':1,'Home':2, 'Business':3, 'Auto':4, 'Other':5}).astype(int)
validation['HasCoSigner']= validation['HasCoSigner'].map({'No':0,'Yes':1}).astype(int)
validation['LoanTerm']= (validation['LoanTerm']/12).astype(int)


# List of columns to be processed for Z-score normalization
val_independent_variables = ['Age','Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', \
                            'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']

# apply Z-score normalization and rounding to 4 decimal places
for column in val_independent_variables:
    validation[column] = scaler.fit_transform(validation[[column]])
    validation[column] = validation[column].apply(lambda x: round(x, 4))


# One hot encoding for categorical variables #columns_encode is at the train data
validation = pd.get_dummies(validation, columns=columns_encode)

# check and save the columns name that boolean values and convert them to binary
validation_columns = validation.columns
for column in validation_columns:
    if validation[column].dtype == bool:
        validation[column] = validation[column].astype(int)


#split validation data into X and Y
X_validation = validation.drop(['Default'], axis=1)
Y_validation = validation['Default']



#drop the independent variables based on VIF values from train
X_validation = X_validation.drop(vif_data['feature'], axis=1)
print(X_validation.columns)

#combine the Xs and Ys back
# export (validation) to csv 
validation = pd.concat([X_validation, Y_validation], axis=1)
validation.to_csv('validation_preprocessed.csv', index=False) 

Index(['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
       'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'HasMortgage',
       'HasDependents', 'HasCoSigner', 'Education_1', 'Education_2',
       'Education_3', 'Education_4', 'EmploymentType_1', 'EmploymentType_2',
       'EmploymentType_3', 'EmploymentType_4', 'LoanPurpose_1',
       'LoanPurpose_2', 'LoanPurpose_3', 'LoanPurpose_4', 'LoanPurpose_5'],
      dtype='object')


In [8]:
#%%script echo skipping           # this line is to skip running this cell. Uncomment if you want to skip this cell
#######Preprocessing test data #######

if 'LoanID' in test.columns:
    test = test.drop(['LoanID'], axis=1)
else:
    print('LoanID column is not present or has removed, check the columns')

test['Education'] = test['Education'].map({'High School':1,'Bachelor\'s':2, 'Master\'s':3, 'PhD':4}).astype(int)
test['EmploymentType'] = test['EmploymentType'].map({'Unemployed':1,'Part-time':2, 'Full-time':3, 'Self-employed':4}).astype(int)
test['MaritalStatus']= test['MaritalStatus'].map({'Single':1,'Married':2, 'Divorced':3}).astype(int)
test['HasMortgage']= test['HasMortgage'].map({'No':0,'Yes':1}).astype(int)
test['HasDependents']= test['HasDependents'].map({'No':0,'Yes':1}).astype(int)
test['LoanPurpose']= test['LoanPurpose'].map({'Education':1,'Home':2, 'Business':3, 'Auto':4, 'Other':5}).astype(int)
test['HasCoSigner']= test['HasCoSigner'].map({'No':0,'Yes':1}).astype(int)
test['LoanTerm']= (test['LoanTerm']/12).astype(int)

# List of columns to be processed for Z-score normalization
test_independent_variables = ['Age','Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', \
                       'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']

# apply Z-score normalization and rounding to 4 decimal places
for column in test_independent_variables:
    test[column] = scaler.fit_transform(test[[column]])
    test[column] = test[column].apply(lambda x: round(x, 4))


# One hot encoding for categorical variables #columns_encode is at the train data
test = pd.get_dummies(test, columns=columns_encode)


#split validation data into X and Y
X_test = test.drop(['Default'], axis=1)
Y_test = test['Default']



#drop the independent variables based on VIF values from train
X_test = X_test.drop(vif_data['feature'], axis=1)
print(X_test.columns)

#combine the Xs and Ys back
# export (test) to csv
test = pd.concat([X_test, Y_test], axis=1)
test.to_csv('test_preprocessed.csv', index=False) 




Index(['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed',
       'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'HasMortgage',
       'HasDependents', 'HasCoSigner', 'Education_1', 'Education_2',
       'Education_3', 'Education_4', 'EmploymentType_1', 'EmploymentType_2',
       'EmploymentType_3', 'EmploymentType_4', 'LoanPurpose_1',
       'LoanPurpose_2', 'LoanPurpose_3', 'LoanPurpose_4', 'LoanPurpose_5'],
      dtype='object')
